# Context and Objectives

## Context

The company provides a feature that allows users to schedule recurring allowances, enabling them to set a specific amount and frequency (e.g., daily, weekly, biweekly, or monthly) for payments they will receive.

Recently, issues were identified in the backend process responsible for updating the allowance and payment schedule tables. These issues led to discrepancies, particularly in the `next_payment_day` fields across the datasets. The data provided reflects all recorded events and the state of the backend tables up to **December 3, 2024**, which should be considered as the current date for analysis.

## Objectives

The objective of this analysis is to investigate the provided datasets and generate a detailed report identifying discrepancies and patterns related to the `next_payment_day` and `payment_date` fields across the backend tables.

Key objectives:

1. **Identify Discrepancies**:
   - Verify if there are any mismatches between the `next_payment_day` from the `allowance_backend_table` and the `payment_date` in the `payment_schedule_backend_table`.
   
2. **Analyze Data Patterns**:
   - Examine recurring issues, such as users having multiple records in the `payment_schedule_backend_table` or incorrect `payment_date` values, and identify potential root causes.
   
3. **Provide Insights**:
   - Highlight potential failures in backend processes that could explain the discrepancies observed.

   ### Data Availability
The tables in RAW format are available in the repository, allowing for direct analysis and verification of the discrepancies observed
https://gist.github.com/DaniModak/d0cdc441bc2cab2abdc5b37e45ca5cb4

In [ ]:
#Imports
import os #For interacting with the operating system, such as accessing file directories
import json #For reading data from the JSON file containing the events.
import pandas as pd #For data manipulation and analysis, converting JSON/CSV data into DataFrames.
from datetime import datetime, timedelta #For manipulating dates and time intervals.

# "Overview of Functions Used in the Code"

The `load_initial_file()` function loads a JSON or CSV file into a Pandas DataFrame:  
- It checks if the file exists. If not, it displays an error message.  
- For JSON, it uses `json.load()` and normalizes it with `pd.json_normalize()`.  
- For CSV, it uses `pd.read_csv()`.  
- It handles exceptions with `try...except`, displaying read errors.  
- It prints the first few rows of the DataFrame and returns it, or `None` if there is an error.


In [ ]:
def load_initial_file(file_path, file_type="json"):
    """
    Loads a JSON or CSV file into a Pandas DataFrame.

    Args:
        file_path (str): Full path to the file.
        file_type (str): Type of file: 'json' or 'csv'.

    Returns:
        pd.DataFrame: DataFrame loaded from the file.
    """
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return None

    try:
        if file_type == "json":
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
            df = pd.json_normalize(data)
        elif file_type == "csv":
            df = pd.read_csv(file_path)
        else:
            print(f"Unknown file type: {file_type}")
            return None

        print(f"\nDataFrame from {file_type.upper()} ({file_path}):")
        print(df.head())
        return df

    except Exception as e:
        print(f"Error loading file {file_path}: {e}")
        return None


The `calculate_next_ocurrence` function calculates the next valid date based on a given starting date, frequency type (weekly, biweekly, daily, or monthly), and a target day. The function uses these inputs to determine the next occurrence of the specified frequency and day, iterating until the calculated date reaches or exceeds a predefined limit (December 3, 2024).

#### Parameters:
- **`start_date`** (`str`, `datetime`, `pd.Timestamp`): The initial date from which the calculation begins. It can be in various formats (e.g., string, datetime object, or pandas Timestamp).
- **`frequency_type`** (`str`): The frequency type for the occurrence. This can be one of the following:
  - `'weekly'`: For weekly occurrences.
  - `'biweekly'`: For biweekly occurrences.
  - `'daily'`: For daily occurrences.
  - `'monthly'`: For monthly occurrences.
- **`target_day`** (`str`): The target day for the occurrence. This can be:
  - A weekday name (e.g., 'Monday', 'Tuesday') for weekly or biweekly frequencies.
  - `'first_day'`: For the first day of the next month.
  - `'fifteenth_day'`: For the fifteenth day of the next month.

#### Returns:
- **`str`**: The day of the next occurrence in the format `dd` (day of the month).

#### Function Behavior:
- **Weekly and Biweekly Frequencies**: The function calculates the next occurrence of the specified weekday (e.g., if `target_day` is 'Monday', it finds the next Monday after the `start_date`). For biweekly occurrences, it skips one additional week before the next occurrence.
- **Daily Frequency**: The function calculates the next day after the `start_date`.
- **Monthly Frequency**: If `target_day` is `'first_day'`, it returns the first day of the next month. If `target_day` is `'fifteenth_day'`, it returns the fifteenth day of the next month. If the `start_date` already passes the target day, it will calculate the occurrence for the next month.
  
The calculation stops when the result exceeds or matches the predefined limit of **December 3, 2024**, ensuring that no occurrence surpasses this date.

In [ ]:
def calculate_next_ocurrence(start_date, frequency_type, target_day):
    """
    The function uses these inputs to determine the next occurrence of the specified frequency and day,
    iterating until the calculated date reaches or exceeds a predefined limit (December 3, 2024).

    Args:
        start_date (str, datetime, or pd.Timestamp): The initial date from which calculations will begin.
        frequency_type (str): The frequency type for the calculation: 'weekly', 'biweekly', 'daily', or 'monthly'.
        target_day (str): The target day for the calculation. For 'weekly' and 'biweekly', provide the day of the week (e.g., 'monday', 'sunday').
                           For 'monthly', use 'first_day' or 'fifteenth_day'.

    Returns:
        str: The last calculated date in the format 'dd', or None if an invalid input is encountered.
    """

    # If start_date is NaT or null, return None to ignore
    if pd.isna(start_date) or start_date == "NaT":
        return None

    # If it's already a datetime, do nothing
    if isinstance(start_date, datetime):
        pass
    # If it's a Timestamp, convert to datetime
    elif isinstance(start_date, pd.Timestamp):
        start_date = start_date.to_pydatetime()
    else:
        try:
            # Remove milliseconds if present
            start_date = start_date.split('.')[0]
            # Try to convert to datetime with time
            start_date = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
        except ValueError:
            # If it fails, try the format without time
            start_date = datetime.strptime(start_date, "%d/%m/%Y")

    # Define the limit date as December 3, 2024
    limit_date = datetime(2024, 12, 3)
    result_date = None

    while True:
        if frequency_type == "weekly" or frequency_type == "biweekly":
            if frequency_type == 'biweekly':
                weekly_increment = 2
            elif frequency_type == 'weekly':
                weekly_increment = 1

            week_days_map = {
                "sunday": 6,
                "monday": 0,
                "tuesday": 1,
                "wednesday": 2,
                "thursday": 3,
                "friday": 4,
                "saturday": 5
            }

            if target_day not in week_days_map:
                return "Invalid day for weekly or biweekly"

            day_of_week = week_days_map[target_day]

            # Calculate the next desired day in the week
            days_until_next = (day_of_week - start_date.weekday()) % 7
            if days_until_next == 0:
                days_until_next = 7 * weekly_increment
            else:
                days_until_next += 7 * (weekly_increment - 1)

            result_date = start_date + timedelta(days=days_until_next)

        elif frequency_type == "daily":
            # For daily, just add one day
            result_date = start_date + timedelta(days=1)

        elif frequency_type == "monthly":
            # If frequency is monthly, adjust based on the day
            if target_day == "first_day":
                # If it's "first day", go to the first day of the next month
                if start_date.month == 12:
                    result_date = datetime(start_date.year + 1, 1, 1)
                else:
                    result_date = datetime(start_date.year, start_date.month + 1, 1)

            elif target_day == "fifteenth_day":
                # If it's "fifteenth day", check the current date
                if start_date.day < 15:
                    # If before or on the 15th, go to the 15th of the same month
                    result_date = datetime(start_date.year, start_date.month, 15)
                else:
                    # If after the 15th, go to the 15th of the next month
                    if start_date.month == 12:
                        result_date = datetime(start_date.year + 1, 1, 15)
                    else:
                        result_date = datetime(start_date.year, start_date.month + 1, 15)
            else:
                return "Invalid day for monthly"

        else:
            return "Invalid frequency type"

        # Update the start_date for the next calculation
        start_date = result_date

        # Check if the limit date has been reached
        if result_date > limit_date:
            break

    # If the last date was on or before the limit, calculate once more
    if result_date <= limit_date:
        if frequency_type == "weekly" or frequency_type == "biweekly":
            result_date += timedelta(weeks=weekly_increment)
        elif frequency_type == "daily":
            result_date += timedelta(days=1)
        elif frequency_type == "monthly":
            if target_day == "first_day":
                if result_date.month == 12:
                    result_date = datetime(result_date.year + 1, 1, 1)
                else:
                    result_date = datetime(result_date.year, result_date.month + 1, 1)
            elif target_day == "fifteenth_day":
                if result_date.month == 12:
                    result_date = datetime(result_date.year + 1, 1, 15)
                else:
                    result_date = datetime(result_date.year, result_date.month + 1, 15)

    return result_date.strftime("%d")

`calculate_incremented_date`: Calculates the next exact date based on the given start date, frequency type (daily, weekly, biweekly, or monthly), and target day. It does not involve any looping but directly computes the next occurrence of the specified date, according to the provided parameters.

#### Arguments:
- **`data_inicial`** (`datetime` or `str`): The initial date to start the calculation. Can be in various formats.
- **`tipo_frequencia`** (`str`): The frequency type. Can be `"daily"`, `"weekly"`, `"biweekly"`, or `"monthly"`.
- **`dia`** (`str`): The target day for the frequency. Can be a weekday (e.g., `"monday"`, `"sunday"`) or specific days like `"first_day"` or `"fifteenth_day"` for monthly frequency.

#### Returns:
- **`str`**: The resulting day of the month in `dd` format.
- If the date can't be calculated (due to invalid parameters), returns an error message (e.g., `"Invalid day for weekly or biweekly"`).

#### Function Workflow:
1. **Initial Date Conversion**:
   - Converts the `data_inicial` into a `datetime` object if it's in string format or a `Timestamp`.
2. **Date Calculation**:
   - Based on the `tipo_frequencia`, calculates the next valid occurrence of the target day:
     - **Weekly**: Finds the next occurrence of the specified weekday.
     - **Biweekly**: Finds the next occurrence of the specified weekday, skipping one full week.
     - **Daily**: Adds exactly one day to the start date.
     - **Monthly**: Calculates the next occurrence of the specified day, either the first day or the fifteenth day of the next month.
3. **No Loops**: The function directly calculates the next date based on the given parameters, without iteration.




In [ ]:
def calculate_incremented_date(start_date, frequency_type, day):
    """
    Calculates the next exact occurrence based on the provided start date, frequency type, and target day.
    This function does not use loops. It directly calculates the next valid date based on the given frequency
    and target day.

    Args:
        start_date (datetime or str): The initial date from which to calculate the next occurrence.
        frequency_type (str): The frequency type, which can be 'weekly', 'biweekly', 'daily', or 'monthly'.
        day (str): The target day to calculate the next occurrence. For 'weekly' or 'biweekly', this is a day of the week (e.g., 'monday').
                   For 'monthly', it can be 'first_day' or 'fifteenth_day'.

    Returns:
        str: The day of the month for the next occurrence in the format 'dd'.
        If the calculation fails or the frequency type is invalid, returns an error message.
    """

    # If start_date is NaT or null, return None to ignore
    if pd.isna(start_date) or start_date == "NaT":
        return None

    # If already a datetime, do nothing
    if isinstance(start_date, datetime):
        pass
    # If it's a Timestamp, convert to datetime
    elif isinstance(start_date, pd.Timestamp):
        start_date = start_date.to_pydatetime()
    else:
        try:
            # Remove milliseconds if any
            start_date = start_date.split('.')[0]
            # Try to convert to the format with time
            start_date = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
        except ValueError:
            # If it fails, try the format without time
            start_date = datetime.strptime(start_date, "%d/%m/%Y")

    result_date = None

    # Handle the frequency calculation
    if frequency_type == "weekly" or frequency_type == "biweekly":
        weekly_increment = 1 if frequency_type == "weekly" else 2

        week_days = {
            "sunday": 6,
            "monday": 0,
            "tuesday": 1,
            "wednesday": 2,
            "thursday": 3,
            "friday": 4,
            "saturday": 5
        }

        if day not in week_days:
            return "Invalid day for weekly or biweekly"

        day_of_week = week_days[day]

        # Calculate the next desired day in the week
        days_until_next = (day_of_week - start_date.weekday()) % 7
        if days_until_next == 0:
            days_until_next = 7 * weekly_increment
        else:
            days_until_next += 7 * (weekly_increment - 1)

        result_date = start_date + timedelta(days=days_until_next)

    elif frequency_type == "daily":
        # For daily, just add one day
        result_date = start_date + timedelta(days=1)

    elif frequency_type == "monthly":
        # If the frequency is monthly, adjust based on the day
        if day == "first_day":
            # If "first day", go to the first day of the next month
            if start_date.month == 12:
                result_date = datetime(start_date.year + 1, 1, 1)
            else:
                result_date = datetime(start_date.year, start_date.month + 1, 1)

        elif day == "fifteenth_day":
            # If "fifteenth day", check the current date
            if start_date.day < 15:
                # If before or on the 15th, go to the 15th of the same month
                result_date = datetime(start_date.year, start_date.month, 15)
            else:
                # If after the 15th, go to the 15th of the next month
                if start_date.month == 12:
                    result_date = datetime(start_date.year + 1, 1, 15)
                else:
                    result_date = datetime(start_date.year, start_date.month + 1, 15)
        else:
            return "Invalid day for monthly"

    else:
        return "Invalid frequency type"

    # Return the date formatted, if it was correctly calculated
    if result_date is not None:
        return result_date.strftime("%d")
    else:
        return "Error in date calculation"

The function `categorize_discrepancy` categorizes discrepancies based on the difference between two timestamps. If the absolute value of the timestamp difference is less than 1 day, the function classifies the discrepancy as `'backend logic issues'`. Otherwise, it categorizes it as `'timestamp delay issue'`. This helps to distinguish between logic errors and delays in the timestamp processing.

In [ ]:
def categorize_discrepancy(row):

    """
    Categorizes discrepancies based on the difference between two timestamps, `timestamp_diff`.
    - If the absolute value of `timestamp_diff` is less than 1 day, classify as 'backend logic issues'.
    - Otherwise, classify as 'timestamp delay issue'.

    Parameters:
    row (pandas.Series): A row of the DataFrame containing the timestamp_diff value.

    Returns:
    str: The category of the discrepancy ('backend logic issues' or 'timestamp delay issue').
    """

    # If the timestamp_diff is less than 1 day, categorize as 'backend logic issues'
    if abs(row['timestamp_diff']) < pd.Timedelta(days=1):
        return 'backend logic issues'
    else:
        return 'timestamp delay issue'

# Code Implementation

Starting using the funtion to load the files for analysys

In [ ]:
# Get the directory where the script is located
file_path = os.path.dirname(os.path.abspath(__file__))

# Build paths for the files
allowance_events_path = os.path.join(file_path, "allowance_events.json")
allowance_backend_path = os.path.join(file_path, "allowance_backend_table.csv")
payment_schedule_path = os.path.join(file_path, "payment_schedule_backend_table.csv")

In [ ]:
# Load files using the generic function
allowance_events_df = load_initial_file(allowance_events_path, "json")
allowance_backend_df = load_initial_file(allowance_backend_path, "csv")
payment_schedule_df = load_initial_file(payment_schedule_path, "csv")




In this section, we ensure that the data types in the three DataFrames (`allowance_events_df`, `allowance_backend_df`, `payment_schedule_df`) are correctly defined:

- **`df_events`**: Converts columns like `'event.name'`, `'user.id'`, and `'allowance.amount'` to appropriate types (`category`, `string`, `float`) and transforms the `'event.timestamp'` to `datetime`.
  
- **`df_backend`**: Converts columns such as `'uuid'` and `'status'` to the correct types and processes `'updated_at'` by converting it to `datetime`, handling missing values and Unix timestamps.

- **`df_payment`**: Converts `'user_id'` to `string` and `'payment_date'` to `int`.

These transformations ensure consistent data types for analysis.

In [ ]:
# Convert data types for allowance_events_df to ensure correct data format
df_events = allowance_events_df.astype({
    'event.name': 'category',
    'user.id': 'string',
    'allowance.amount': 'float',
    'allowance.scheduled.frequency': 'category',
    'allowance.scheduled.day': 'string'
})

# Convert 'event.timestamp' to datetime format
df_events['event.timestamp'] = pd.to_datetime(allowance_events_df['event.timestamp'], format='%Y-%m-%d %H:%M:%S')

#Print to check if the data was correctly hadled
print(df_events.dtypes)
print(df_events.head())

In [ ]:
# Convert data types for allowance_backend_df to ensure correct data format
df_backend = allowance_backend_df.astype({
    'uuid': 'string',
    'creation_date': 'string',
    'frequency': 'category',
    'day': 'string',
    'next_payment_day': 'int',
    'status': 'category'
})

# Convert 'updated_at' to datetime format and remove timezone information
df_backend['updated_at'] = pd.to_datetime(allowance_backend_df['updated_at'], errors='coerce').dt.tz_localize(None)

# Check where conversion failed (values as 'NaT')
print(df_backend[df_backend['updated_at'].isna()])

# Convert Unix timestamp values to datetime and remove timezone as well
df_backend['updated_at'] = df_backend['updated_at'].fillna(
    pd.to_datetime(allowance_backend_df['updated_at'], unit='s', errors='coerce').dt.tz_localize(None)
)

# Verify the data types in the backend DataFrame and display the first few rows
print(df_backend.dtypes)
print(df_backend.head())

In [ ]:
# Convert data types for payment_schedule_df to ensure correct data format
df_payment = payment_schedule_df.astype({
    'user_id': 'string',
    'payment_date': 'int'
})

# Verify the data types in the backend DataFrame and display the first few rows
print(df_payment.dtypes)
print(df_payment.head())

#Analysis of Disabled and Duplicate Users in Events and Backend Tables

In this section, we analyze the presence of disabled and duplicate users in the `df_events` and `df_backend` tables.


In [ ]:
#Count users with 'disabled' status in backend table
disabled_users_backend = df_backend[df_backend["status"] == "disabled"]
num_disabled_users_backend = disabled_users_backend.shape[0]

# Display the number of users with 'disabled' status
print(f'Number of users with "disabled" status in the allowance_backend_table: {num_disabled_users_backend}')

# Calculate the percentage of disabled users
total_users_backend = df_backend.shape[0]
percentage_disabled_users = (num_disabled_users_backend / total_users_backend) * 100

# Step 2: Print the users with 'disabled' status in allowance_backend_table
print("List of disabled users in the allowance_backend_table:")
print(disabled_users_backend)

Now, we need to remove these users from allowance_backend_table (`df_backend`) making sure that we will only use enables allowances

In [ ]:
# Step 2: Remove users with 'disabled' status from the dataframe
df_backend_enabled_users = df_backend[df_backend["status"] != "disabled"].reset_index(drop=True)

# Display the dataframe without the disabled users
print(df_backend_enabled_users.head())


Now, let's analyze and remove the disabled users from the `df_events` dataframe to ensure consistency in the analysis by eliminating discrepancies caused by the presence of these users.

First, we count the number of disabled users present in the `df_events` (allowance_events_table)

In [ ]:
# Merge the df_backend and df_events tables
df_merged = pd.merge(df_backend, df_events, left_on='uuid', right_on='user.id', how='inner')

# Filter users with 'disabled' status in the df_merged table
disabled_users_events = df_merged[df_merged["status"] == "disabled"]

# Count the number of disabled users present in the events table
num_disabled_users_events = disabled_users_events["user.id"].nunique()

# Display the number of disabled users present in the events table
print(f'Number of disabled users present in the events table: {num_disabled_users_events}')

and then we remove these users from `df_events`(allowance_events_table)

In [ ]:
#Get the user IDs of disabled users present in the events table
disabled_user_ids_in_events = disabled_users_events["user.id"].unique()

#Remove these users from the df_events table and reset the index of the resulting dataframe to ensure a clean, sequential index after filtering
df_events_cleaned = df_events[~df_events["user.id"].isin(disabled_user_ids_in_events)].reset_index(drop=True)

# Checking number of users before removal
num_users_before = len(df_events)

# Checking number of users after removing the disabled ones
num_users_after = len(df_events_cleaned)

#Calculationg number of users removed
num_users_removed = num_users_before - num_users_after

# Display the number of removed users
# The number of users removed in the df_events table is higher because a single user can be associated with multiple events.
print(f"Number of users removed: {num_users_removed}")

# Display the cleaned dataframe
print(f"The cleaned dataframe after removing disabled users is:")
print(df_events_cleaned.head())

# Comparative Analysis: Event-Based Payment Dates vs. Backend System Dates

In this section, we will calculate the correct payment date based on the event data table (considered the source of truth). We will then compare these calculated payment dates with the dates in the backend system. If the dates match, it will indicate that the backend system is accurate. If there is a discrepancy, further analysis will be conducted to identify the root cause of the error.


In first step, we will calculate the correct payment date based on the `event.timestamp` and `allowance.scheduled.frequency` fields from the cleaned events table (without disabled users), along with the `allowance.scheduled.day`. We will use the `calculate_next_occurrence` function to determine the next correct payment date according to the specified frequency and timestamp.








In [ ]:
# Calculating the 'next_correct_allowance_day' column and adding it to df_events_cleaned
df_events_cleaned["next_expected_payment_date"] = df_events_cleaned.apply(
    lambda row: calculate_next_ocurrence(
        row["event.timestamp"],
        row["allowance.scheduled.frequency"],
        row["allowance.scheduled.day"]
    ),
    axis=1
)

# Displaying the results with new column
print(df_events_cleaned[["user.id", "event.timestamp", "next_expected_payment_date"]])

It is essential to retain only the most recent modification `event.timestamp` for each user in `df_events_cleaned`to simplify the comparison process. In the backend table, we store only the latest record for each usery, unlike the events table. So, as the next step, I will sort the data and retain only the most recent record for each user in `df_events_cleaned`

In [ ]:
##### Filtering to keep only the latest entry per user #####

# Sort by user.id and event.timestamp to ensure the latest event for each user is at the top
df_events_cleaned = df_events_cleaned.sort_values(by=["user.id", "event.timestamp"], ascending=[True, False])

# Remove duplicates, keeping only the latest event for each user
df_events_cleaned = df_events_cleaned.drop_duplicates(subset="user.id", keep="first").reset_index(drop=True)

# Display the filtered dataframe
print("Filtered allowance_ events dataFrame with the latest event per user:")
print(df_events_cleaned.head())

Now, with only the latest records, I can begin comparing the `next_expected_payment_date` from the `df_events_cleaned`, which was calculated using the `calculate_next_occurrence` function, with the `next_payment_day` field from the `allowance_backend` table."

First, we do the merge between the two dataframes to bring the fields:

In [ ]:
# Merging df_events_cleaned with df_backend
df_events_merged = pd.merge(
    df_events_cleaned,
    df_backend[['uuid', 'creation_date', 'frequency', 'day', 'updated_at', 'next_payment_day']],
    left_on='user.id',    # Key from df_events_cleaned
    right_on='uuid',      # Key from df_backend
    how='left'            # To keep all events, even those without a match in the backend
)

# Displaying the final result with the relevant columns
print("Filtered allowance_ events dataFrame with the latest event per user:")
print(df_events_merged.head())

To handle data type differences, we convert next_payment_day to remove the decimal part:

In [ ]:
df_events_merged['next_payment_day'] = df_events_merged['next_payment_day'].fillna(0).astype(int).astype(str).str.zfill(2)

# Displaying results after conversion
print(df_events_merged.head())

Now that types are the same, we can use the calculated field `next_expected_payment_date` to compare with `next_payment_day`from backend. In scope of this, I will create a new field called `is_next_payment_day_correct` with results TRUE or FALSE:

In [ ]:
# Creating new column with comparison results
df_events_merged['is_next_payment_day_correct'] = (
    df_events_merged['next_expected_payment_date'] == df_events_merged['next_payment_day']
)

After adding this column, we now have information about whether the `next_expected_payment_date` matches `the next_payment_day` from the backend.

If the result is `False`, it indicates that the payment dates are being calculated incorrectly. This could be due to various factors, which I will investigate further in the next steps.

If the result is `True`, it could indicate two possibilities:


1.   The `next_payment_day` is correct because the user was properly synchronized with backend
2.   The `True` result was generated by coincidence, with no correlation to the correct timestamp



In [ ]:
#Displaying results for first visual analysis
print(df_events_merged.head(30))


After presenting these results for a preliminary analysis, it becomes clear that the majority of cases return False, and this is correlated with the timestamp difference between the events table and the backend table.

In this case, to avoid performing numerous calculations with a date field (which can easily lead to discrepancies), I realized, after doing a quick manual calculation, that in some cases the next_payment_date field is being calculated based on the timestamp from the backend table (instead of the events table). This leads to incorrect calculations because most of the events are not synchronized at the same time they enter the events table, so they are added to the backend table with some delays.

In [ ]:
# Converting the columns to datetime format, in case they are not already in that format
df_events_merged['event.timestamp'] = pd.to_datetime(df_events_merged['event.timestamp'])
df_events_merged['updated_at'] = pd.to_datetime(df_events_merged['updated_at'])

# Calculating the timestamp difference between the backend and events tables
df_events_merged['timestamp_diff'] = df_events_merged['event.timestamp'] - df_events_merged['updated_at']

# Displaying the result to verify the timestamp difference
print("df_events_merged with new column timestamp_diff:")
print(df_events_merged.head())

After calculating the time difference between the synchronization of the events table and the backend, and after performing a quick manual check, I noticed that in some cases, the next_payment_day in the backend table is calculated based on the timestamp from the backend (which may differ from the timestamp in the events table), leading to errors in the calculation of the next_payment_day.

To be certain, I decided to calculate a new field using the `calculate_incremented_date` function, where I will base the calculation on the updated_at field to determine the exact next payment day, without considering the loop that should be performed until today to identify the correct next payment day, as we did in the other function

In [ ]:
df_events_merged["next_payment_day_from_updated_at"] = df_events_merged.apply(
    lambda row: calculate_incremented_date(
        row["updated_at"],
        row["frequency"],
        row["day"]
    ),
    axis=1
)

# Displaying new column next_payment_day_from_updated_at in dataframe:
print("Displaying new column next_payment_day_from_updated_at in dataframe:")
print(df_events_merged[['user.id', 'creation_date', 'frequency', 'day', 'updated_at', 'next_payment_day']])

Now, I will compare with `next_payment_day`from backend to ensure that my theory is right for such cases:

In [ ]:
# Comparison between fields 'next_payment_day' from backend and new calculated field 'next_payment_day_from_updated_at'
df_events_merged['match_with_updated_at'] = (
    df_events_merged['next_payment_day'] == df_events_merged['next_payment_day_from_updated_at']
)

# Displaying new column next_payment_day_from_updated_at in dataframe:
print("Displaying new column 'match_with_updated_at' in dataframe:")
print(df_events_merged.head())

From a superficial analysis of the printed results with `True` or `False`, it is evident that, in some cases, the `next_payment_day` is being calculated using the `updated_at field`. This field, in most cases, does not synchronize immediately with the user’s events table, leading to errors in the calculation of the `next_payment_date` and also says that the logic for this field in backend is not doing the looping to reach frequencies till current day in calculations.

# Analysis of Payment Date Discrepancies in `allowance_backend_table` : Identifying Patterns and Hypotheses

Now that we have calculated auxiliary fields such as match_with_updated_at, which allows us to identify errors and their types by comparing with next_payment_day, and next_expected_payment_date, which shows the actual date when the payment should occur, we can begin our quantitative analysis and report from backend team.

### First analysis:

*Filtering the rows where match_with_payment_next_day is True and is_next_payment_day_correct is also True.*

These combined conditions mean that the code is filtering the records where both payment date validation criteria are true, that is, where the system correctly calculated the next payment day and this calculation matches the expected one.

In [ ]:
# Filtering
filtered_rows = df_events_merged[(df_events_merged['match_with_updated_at'] == True) &
                                   (df_events_merged['is_next_payment_day_correct'] == True)]

# Calculating the number of filtered rows
count_filtered_rows = len(filtered_rows)

# Calculating the percentage relative to the total number of rows in the DataFrame
percentage = (count_filtered_rows / len(df_events_merged)) * 100

# Displaying the result of counting and percentage
print(f"Percentage of rows where match_with_updated_at and is_next_payment_day_correct are both TRUE: {percentage:.2f}%")
print(f"Total number of records where both criteria are TRUE: {count_filtered_rows}")

# Display the rows where both conditions are TRUE
print("Rows where both match_with_updated_at and is_next_payment_day_correct are TRUE:")
print(filtered_rows.head()) #adjust here if you want to see all filtered_rows

If the values in the calculated columns and the payment_next_day column match, it indicates that there are no discrepancies in the data. Therefore, we can exclude these records from the analysis moving forward, as they are considered correct and consistent

In [ ]:
# Removing rows where 'match_with_updated_at' is True and 'is_next_payment_day_correct' is True
df_events_adjusted = df_events_merged[~((df_events_merged['match_with_updated_at'] == True) &
                                      (df_events_merged['is_next_payment_day_correct'] == True))]

# Displaying the cleaned DataFrame
print("Displaying dataframe without correct next_payment_day to focus on discrepancies:")
print(df_events_adjusted.head())

### Second analysis:

Filtering the rows where `match_with_updated_at` is `True` and `is_next_payment_day_correct` is `False`.

This means that the rows that match with match_with_updated_at but do not match with `is_next_payment_day_correct` are taking into account the `updated_at` timestamp to calculate the `next_payment_day`. This indicates that either the logic of the loop to calculate the nextpayment day, considering the current date, is inconsistent, or that the `updated_at` timestamp has a delay.

In [ ]:
# Filtering the rows where match_with_payment_next_day is True and is_next_payment_day_correct is False
filtered_rows_2 = df_events_merged[(df_events_merged['match_with_updated_at'] == True) &
                                   (df_events_merged['is_next_payment_day_correct'] == False)]

# Calculating the number of filtered rows
count_filtered_rows_2 = len(filtered_rows_2)

# Calculating the percentage relative to the total number of rows in the DataFrame
percentage = (count_filtered_rows_2 / len(df_events_merged)) * 100

# Display the rows where both conditions are TRUE
print("Rows where both match_with_updated_at and is_next_payment_day_correct are TRUE:")
print(filtered_rows_2.head()) #adjust here if you want to see all filtered_rows


If the values in the calculated columns and the payment_next_day column match, it indicates that there are discrepancies in the data. Both caused by delays in sync with events table or because the logic is failing to calculate the next_payment_day

In [ ]:

# Adding reason_of_discrepancy only to rows in filtered_rows_2
df_events_adjusted['reason_of_discrepancy'] = df_events_adjusted.apply(
    lambda row: categorize_discrepancy(row) if row.name in filtered_rows_2.index else '', axis=1)

# Count the number of rows for each category
category_counts = df_events_adjusted['reason_of_discrepancy'].value_counts()

# Print the counts for each category
print("Count of rows categorized by reason_of_discrepancy:")
print(category_counts)

# Print rows for each category
for category in category_counts.index:
    print(f"\nRows categorized as '{category}':")
    print(df_events_adjusted[df_events_adjusted['reason_of_discrepancy'] == category][['user.id', 'reason_of_discrepancy']].head())

# Backend Analysis Report
After categorizing the discrepancies based on the timestamp differences and analyzing the reason_of_discrepancy, we observe the following:

Backend Logic Issues: The discrepancies categorized under "backend logic issues" indicate that the timestamp difference is less than one day, which suggests potential issues with how the backend is handling date logic.

Timestamp Delay Issues: On the other hand, discrepancies under the "timestamp delay issue" category indicate that the issue is related to a delay in the updated_at timestamp, which could be affecting the payment day calculations.


1.   **Backend Logic Issues:** The discrepancies categorized under "backend logic issues" indicate that the timestamp difference is less than one day, which suggests potential issues with how the backend is handling date logic
2.   **Timestamp Delay Issues:** On the other hand, discrepancies under the "timestamp delay issue" category indicate that the issue is related to a delay in the updated_at timestamp, which could be affecting the payment day calculations


**25 Rows Reporting Unknown Issues**: Among the categorized rows, we have 25 entries that are categorized as having an unknown issue. This discrepancy needs to be further investigated to determine its root cause and resolve it effectively.

In [ ]:
# Saving the final dataframe with the new 'reason_of_discrepancy' column to a CSV file
output_file = 'discrepancies_in_payment_dates.csv'
df_events_adjusted.to_csv(output_file, index=False)

# Confirming the file is saved
print(f"\nThe final DataFrame has been saved to {output_file}")

There are two users who do not have a UUID created in the Backend table

1.   1cf825ad-c6fc-4881-9df1-20b495f375d8
2.   f0c58b79-2e41-4487-970f-fe6206bbe20b

And also we can find an extra user User.Id/UUID = X, clearly indicating an error.

# Payment Schedule Backend Table analysis

In addition to removing duplicates from the `user_id` field, the analysis also checks if the `payment_date` field from the `df_payment` table matches the `next_payment_day` field from the backend table and also `next_expected_payment_date`from events. In other words, it compares the payment dates between the three tables to see if they align for each user.

Essentially, I am performing two analyses:

Removing duplicates: Ensuring that each user_id in the df_payment table is unique.
Verifying date consistency: Validating if the payment dates in df_payment are correct according to the information from the backend.

In [ ]:
# Counting how many times each user_id appears
user_id_counts = df_payment['user_id'].value_counts()

# Filtering only the user_ids that appear more than once
duplicate_user_ids = user_id_counts[user_id_counts > 1]

print(f"Number of duplicate user_ids: {len(duplicate_user_ids)}")
print(duplicate_user_ids)

# Removing duplicates and keeping only the first occurrence of each user_id
df_payment_unique = df_payment.drop_duplicates(subset='user_id', keep='first')

# Verifying the result
print(f"Dataframe after removing duplicates: {df_payment_unique.head()}")

As next step, I bring again the table `df_events_merged`to compare datas between fields `payment_date`(df_payment), `next_payment_day`(backend) and `next_expected_payment_date`(events)

In [ ]:
df_final_merged = pd.merge(
    df_events_merged,  # DataFrame df_merged (with backend and user_id links)
    df_payment,        # DataFrame df_payment
    left_on='user.id', # Key from df_merged
    right_on='user_id',# Key from df_payment
    how='left',        # Left join to retain all records from df_merged
    suffixes=('_merged', '_payment')  # Suffixes for column names after merge
)

# Displaying the resulting DataFrame to verify the merge outcome
print(df_final_merged.head())

The purpose of bringing in these fields aligns with the following logic:

1. **'backend error - logic'**: When payment_date (df_payment) matches next_payment_day(backend) but does not match next_expected_payment_date(events), indicating a logic error, but means that the date is sync with backend.
2. **'backend error - timestamp'**: When payment_date matches next_expected_payment_date but does not match next_payment_day, suggesting that the logic is correct with events
3. **'unknown error'**: When payment_date matches neither next_payment_day nor next_expected_payment_date, signaling an unknown error or discrepancy.
4. **'correct payment date'**: When payment_date matches both next_payment_day and next_expected_payment_date, indicating the backend logic and timestamp are correct for payment.





In [ ]:
# Creating a new column to categorize the payment date matches
df_final_merged['payment_date_status'] = df_final_merged.apply(
    lambda row: 'backend error - logic' if row['payment_date'] == row['next_payment_day'] and row['payment_date'] != row['next_expected_payment_date'] else
               ('backend error - timestamp' if row['payment_date'] != row['next_payment_day'] and row['payment_date'] == row['next_expected_payment_date'] else
               ('unknown error' if row['payment_date'] != row['next_payment_day'] and row['payment_date'] != row['next_expected_payment_date'] else
               'correct payment date')), axis=1)

# Display the result
print(df_final_merged[['user_id', 'payment_date', 'next_payment_day', 'next_expected_payment_date', 'payment_date_status']].head())


As the final result, I export in CSV file for backend team analysis:

In [ ]:
# Save the result to a CSV file
output_file_payment_status = 'payment_table_discrepancy.csv'
df_final_merged.to_csv(output_file_payment_status, index=False)